In [21]:
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import pandas as pd
import pprint
import getpass

from datetime import datetime
from time import sleep
from random import SystemRandom

try:
    import robin_stocks.robinhood as r
except ImportError:
    pass

In [25]:
RAND = SystemRandom()
ACTIONS = {1: [0.6, 0.4],
           2: [0.57, 0.43],
           3: [0.65, 0.35],
           4: [0.75, 0.25],
           5: [0.48, 0.52],
           6: [0.6, 0.4],
           8: [0, 1],
           9: [0.7, 0.3],
           10: [0.53, 0.47],
           13: [0.76, 0.24],
           14: [0.76, 0.24],
           15: [0.65, 0.35],
           17: [0.64, 0.36],
           18: [0.59, 0.41],
           19: [1, 0],
           22: [0.14, 0.86],
           23: [0.67, 0.33],
           25: [0, 1],
           26: [0.7, 0.3],
           27: [0.71, 0.29],
           28: [0.27, 0.44],
           29: [0.23, 0.4],
           30: [0.33, 0.33],
           32: [0.4, 0.4],
           33: [0.26, 0.53],
           36: [0.6, 0.3],
           37: [0.19, 0.55],
           38: [0, 0.6],
           40: [0.3, 0.41],
           41: [0.27, 0.5],
           42: [0.21, 0.46],
           43: [1, 0],
           44: [0.29, 0.57],
           45: [0.29, 0.47],
           46: [0, 0.67],
           49: [0.19, 0.38],
           50: [0.15, 0.62],
           52: [0, 0.5],
           53: [0.42, 0.46],
           54: [0.25, 0.44],
           55: [0, 0.3],
           56: [0, 0.33],
           57: [0, 0],
           59: [0, 0],
           60: [0, 0.25],
           63: [0, 1],
           64: [0, 0.4],
           65: [0, 0.33],
           67: [0, 0.38],
           68: [0, 0.35],
           69: [0, 0.35],
           72: [0, 0.44],
           73: [0, 0.32],
           74: [0, 0.5],
           76: [0, 0.36],
           77: [0, 0.38],
           78: [0, 0.64],
           79: [0, 0.48],
           80: [0, 0.36],
           81: [0, 0.42]}

In [2]:
username = 'watsona4@gmail.com'
password = getpass.getpass()

 ····


In [3]:
r.login(username, password)

NameError: name 'r' is not defined

In [56]:
h = r.get_crypto_historicals('BTC', interval='5minute', span='week')

ERROR: Span must be "hour","day","week","month","3month","year",or "5year"


In [53]:
df = pd.DataFrame(h)
df['begins_at'] = pd.to_datetime(df['begins_at']).dt.tz_localize(None)

In [54]:
df.to_csv('btc_5minute_week_20210421.csv')

In [35]:
df = pd.read_csv('btc_5minute_week_20210421.csv', usecols=['begins_at', 'open_price'], parse_dates=['begins_at'])

In [41]:
df.tail()

,begins_at,open_price
2011,2021-04-21 20:05:00,62146.620
2012,2021-04-21 20:10:00,62258.625
2013,2021-04-21 20:15:00,62131.995
2014,2021-04-21 20:20:00,62216.685
2015,2021-04-21 20:25:00,62239.930


In [57]:
r.get_crypto_quote('BTC')

{'ask_price': '54896.734322',
 'bid_price': '54849.110000',
 'mark_price': '54872.922161',
 'high_price': '56565.285000',
 'low_price': '54197.898876',
 'open_price': '55306.480000',
 'symbol': 'BTCUSD',
 'id': '3d961844-d360-45fc-989b-f6fca761d511',
 'volume': '0.000000'}

In [53]:
i = 0
def get_next_price():
    global i
    i += 1
    #time.sleep(300)
    #quote = r.get_crypto_quote('BTC')
    #return {'time': etime(datetime.now()), 'price': float(quote['ask_price'])}
    return {'time': etime(df.iloc[i-1]['begins_at']), 'price': df.iloc[i-1]['open_price']}

def etime(x):
    return (x - datetime(1900, 1, 1)).total_seconds()/3600/24

In [54]:
def get_action(box):
    b = ACTIONS[box]
    rand = RAND.random()
    if rand < b[0]:
        return 'BUY'
    if rand < b[0] + b[1]:
        return 'HOLD'
    return 'SELL'

In [55]:
def buy(shares, value, price):
    if value == 0:
        return shares, value
    #order = r.order_buy_crypto_by_price('BTC', value)
    #return float(order.quantity)
    return value/price, 0

def sell(shares, value, price):
    if shares == 0:
        return shares, value
    #order = r.order_sell_crypto_by_quantity('BTC', shares)
    #return float(order.price)
    return 0, shares*price

In [71]:
def trade():
    global i
    i = 0
    last_act = 'SELL'
    shares = 0
    value = 100

    p0 = pp1 = pp2 = pp3 = None

    while True:

        p1 = p0

        try:
            p0 = get_next_price()
        except IndexError:
            shares, value = sell(shares, value, p0['price'])
            return value

        if p1 is None: continue

        pp1old = pp1
        pp1 = (p0['price'] - p1['price'])/(p0['price']*(p0['time'] - p1['time']))

        if pp1old is None: continue

        pp2old = pp2
        pp2 = (pp1 - pp1old)/(500*(p0['time'] - p1['time']))

        if pp2old is None: continue

        pp3old = pp3
        pp3 = (pp2 - pp2old)/(500*(p0['time'] - p1['time']))

        if pp3old is None: continue

        pp4 = (pp3 - pp3old)/(500*(p0['time'] - p1['time']))

        def boxpp(x):
            if x < -0.2:
                return 0
            if x < 0.2:
                return 1
            return 2

        box = 27*boxpp(pp1) + 9*boxpp(pp2) + 3*boxpp(pp3) + boxpp(pp4) + 1

        action = get_action(box)

        if action == 'BUY' and last_act != 'BUY':
            shares, value = buy(shares, value, p0['price'])
        elif action == 'SELL' and last_act != 'SELL':
            shares, value = sell(shares, value, p0['price'])
        else:
            last_act = 'HOLD'

        #print(f'action={action:4s}, shares={shares:.4f}, value={value:.2f}')
        
    return value

In [78]:
vals = []
for j in range(20):
    vals.append(trade()/100 - 1)

In [79]:
avg = np.mean(vals)
stdev = np.std(vals)

In [84]:
ten = 1/np.log10(1 + avg)
pm = stdev*ten**2/(1 + avg)
print(f'{ten} +/- {pm} weeks')
print(f'{6*ten} +/- {6*pm} weeks')

4.1656473549394555 +/- 0.7598550037108482 weeks
24.993884129636733 +/- 4.559130022265089 weeks
